In [80]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pds
import pathlib 
import math
import cmath
from scipy import stats
from scipy.constants import k
from copy import deepcopy
import requests
from bs4 import BeautifulSoup
import pandas as pd
from geopy.geocoders import Nominatim
import random
import folium
from datetime import datetime
import time
import responses
import googlemaps
import numpy as np
from sklearn.cluster import KMeans
%matplotlib inline
import pickle

In [32]:
with open("API_key.txt", "r") as f:
    GMAP_API_KEY = f.read()
"""
This calls the Google Maps API on all pairwise combinations of a supplied list of M addresses, and computes an MxM matrix for distances and times.

Input: List of addresses

Output: Distance Matrix, Time Matrix, in seconds and in meters
"""

def getGoogleTimeDistance(addresses):
    gmaps = googlemaps.Client(key=GMAP_API_KEY)

    curDate = datetime.now()

    time_matrix = np.array([[0] * len(addresses)] * len(addresses))
    distance_matrix = np.array([[0] * len(addresses)] * len(addresses))

    # Iterate through all possible address combinations.
    for i in range(0, len(addresses)):
        for j in range(0, len(addresses)):

            # Do not calculate distance for the same address
            if i == j:
                continue
           
            # Call Google API and get response.
            result = gmaps.distance_matrix(addresses[i], addresses[j], mode='driving', units='metric',traffic_model='best_guess', departure_time=curDate)

            # Index to relevant part of response JSON File.
            curElement = result['rows'][0]['elements'][0]

            # Update matrices
            distance_matrix[i][j] = curElement['distance']['value']
            time_matrix[i][j] = curElement['duration']['value']

    return time_matrix, distance_matrix

def coords_from_address(address):
    """
    inputs: str address
    outputs: tuple[float]  result_loc
    description: converts address to into latitude and longitude
    """
    geolocator = Nominatim(user_agent = "test")
    location = geolocator.geocode(address)
    result_loc = (location.latitude, location.longitude)
    return result_loc

def write_gps_to_csv(gps_list, filename):
    df = pd.DataFrame(gps_list, columns=['Latitude', 'Longitude'])
    df.to_csv(filename, index=False)
    
#write_gps_to_csv(all_coords, "coords.csv")

In [33]:
coords = []
addrs = [
    "2701 Riverside Dr, Ottawa",
    "2515 Bank St, Ottawa",
    "30 Haxby Private, Ottawa",
    "116 Sai Crescent",
    "214 Plymouth St, Ottawa",
    "266 Lorry Greenberg Dr, Ottawa",
    "800 King Edward Ave, Ottawa",
    "2210 Bank St, Ottawa",
    "2161 Thurston Dr, Ottawa",
    "23 Ella Street, Ottawa",
    "11 Oblats Avenue, Ottawa",
    "422 Byron Avenue, Ottawa",
    "1680 Fisher Avenue, Ottawa",
    "825 Cahill Drive, Ottawa",
    "1125 Colonel By Dr, Ottawa",
    "54 Hawk Crescent, Ottawa",
    "1380 Prince of Wales Drive, Ottawa",
    "75 Laurier Ave. E, Ottawa, Ontario, Canada",
    "14 Waller St, Ottawa, Ontario, Canada",
    "363 Rideau St, Ottawa, Ontario, Canada",]

for addr in addrs:
    print(addr)
    coords.append(coords_from_address(addr))
    time.sleep(1)




2701 Riverside Dr, Ottawa
2515 Bank St, Ottawa
30 Haxby Private, Ottawa
116 Sai Crescent
214 Plymouth St, Ottawa
266 Lorry Greenberg Dr, Ottawa
800 King Edward Ave, Ottawa
2210 Bank St, Ottawa
2161 Thurston Dr, Ottawa
23 Ella Street, Ottawa
11 Oblats Avenue, Ottawa
422 Byron Avenue, Ottawa
1680 Fisher Avenue, Ottawa
825 Cahill Drive, Ottawa
1125 Colonel By Dr, Ottawa
54 Hawk Crescent, Ottawa
1380 Prince of Wales Drive, Ottawa
75 Laurier Ave. E, Ottawa, Ontario, Canada
14 Waller St, Ottawa, Ontario, Canada
363 Rideau St, Ottawa, Ontario, Canada


In [ ]:
#T_mat, D_mat = getGoogleTimeDistance(addrs)


In [36]:
#pickle
with open('T_mat','wb') as f1: 
    pickle.dump(T_mat, f1)

with open('T_mat','rb') as f2:
    new_T_mat = pickle.load(f2)
    
with open('D_mat','wb') as f3: 
    pickle.dump(D_mat, f3)

with open('D_mat','rb') as f4:
    new_D_mat = pickle.load(f4)

In [41]:
#exclude index zero because that is caanda post HQ





In [58]:
coords_array = np.array(delivery_coords)
n_trucks = 3
kmeans = KMeans(n_clusters=n_trucks).fit(coords_array) 


cluster_map = pd.DataFrame()
cluster_map['destination_index'] = np.arange(len(coords_array))
cluster_map['cluster'] = kmeans.labels_

In [158]:
def assign_destinations(delivery_coordinates, addresses, n_trucks):
    """
    inputs: list(tuple) delivery coordinates, int n_trucks
    outputs: dict[list[int]] Allocation
    determines which trucks go where, referencing index of original address list and D_mat
    """
    coords_array = np.array(delivery_coords)
    kmeans = KMeans(n_clusters=n_trucks).fit(coords_array) 
    cluster_map = pd.DataFrame()
    #add one because the start location isnt counted
    cluster_map['destination_index'] = np.arange(len(coords_array))
    cluster_map['cluster'] = kmeans.labels_
    
    Allocation = {}
    for c_idx in set(cluster_map["cluster"]):
        location_details = []
        
        destination_idxs = list(cluster_map[cluster_map["cluster"]==c_idx]["destination_index"])
        for i in destination_idxs:
            
            loc_detail = (i+1, delivery_coordinates[i], addresses[i])
            
            location_details.append(loc_detail)
            
        Allocation[c_idx] = location_details
        #print(Allocation)
        
    return Allocation

def sub_DT_mats(Allocation):
    T_mats = {}
    D_mats = {}
    for t in Allocation.keys():
        
        addresses = []
        for loc_detail in Allocation[t]:
            addresses.append(loc_detail[2])
        T_mat, D_mat = getGoogleTimeDistance(addresses)
        T_mats[t] = T_mat
        D_mats[t] = D_mat
        
        
    return T_mats, D_mats
    #extract list of addresses from allocation

def initial_configs(Allocation):
    configs = {}
    

In [121]:
delivery_coords = coords[1:]
addresses = addrs[1:]
destination_assignment = assign_destinations(delivery_coords, addresses, 3)

In [133]:
destination_assignment = assign_destinations(delivery_coords, addresses, 3)

In [137]:
destination_assignment[0][0]

(1, (45.354957, -75.64506230308237), '2515 Bank St, Ottawa')

In [159]:
T_mats, D_mats = sub_DT_mats(destination_assignment)

In [162]:
T_mats

{0: array([[  0, 359, 494, 344, 307, 435, 409, 405],
        [346,   0, 443, 184, 348, 279, 551, 547],
        [472, 418,   0, 385, 608, 297, 709, 705],
        [245, 176, 434,   0, 381, 237, 482, 478],
        [308, 346, 641, 426,   0, 521, 335, 332],
        [413, 275, 335, 243, 518,   0, 650, 646],
        [375, 560, 709, 558, 403, 650,   0, 207],
        [394, 578, 727, 577, 421, 668, 239,   0]]),
 1: array([[  0, 769, 311, 547, 718, 767, 782],
        [280,   0, 420, 224, 312, 541, 392],
        [284, 716,   0, 495, 665, 765, 803],
        [428, 405, 518,   0, 354, 583, 491],
        [407, 127, 548, 351,   0, 301, 223],
        [494, 422, 635, 430, 369,   0, 238],
        [599, 319, 739, 543, 266, 165,   0]]),
 2: array([[  0, 691, 911, 716],
        [678,   0, 557, 252],
        [881, 528,   0, 368],
        [785, 307, 503,   0]])}

{0: {...}, 1: {...}, 2: {...}}

In [ ]:
def find_clusters(detections, next_diff_TH, prev_diff_TH):
    X = np.array(detections)
    inertias = []
    convergence = False
    i=0
    while convergence == False:
        i+=1
        kmeans = KMeans(n_clusters=i, random_state=0, max_iter=5).fit(X) 
        inertia = kmeans.inertia_
        inertias.append(inertia)

        if len(inertias)>3:
            next_difference = abs((inertias[-1]-inertias[-2])/inertias[-2])
            prev_difference = abs((inertias[-2]-inertias[-3])/inertias[-3])
            if next_difference < next_diff_TH and prev_difference > prev_diff_TH:
                convergence = True
    n_clusters = i-1
    kmeans = KMeans(n_clusters=n_clusters, random_state=0, max_iter=5).fit(X)
    return (kmeans.cluster_centers_)

In [25]:
def cost_function(start_idx, end_idx, distance_matrix, lookup_tables):
    """
    inputs: int start_idx, int end_idx, nparray distance_matrix, list[str] lookup_tables
    outputs: float cost
    description: calculates the cost to go from start_idx to end_idx. These idx correspond to distance matrix.
    cost also factors in weather and traffic, information could be obtained from a lookup table
    lookup_tables is filepath to csv file, which would be read to generate our lookup table
    """
    pass

def cost_matrix(distance_matrix, time_matrix, lookup_tables):
    """
    inputs: nparray distance_matrix, nparray time_matrix, str lookup_tables 
    outputs: nparray cost_mat
    description: returns an array that calculates the cost to go from one node to another, for all nodes
    """
    pass

def cluster_coords(coordinates, N_trucks):
    """
    inputs: list coordinates, int N_trucks
    outputs: list[list] grouped_coords
    description: groups the coordinates into n_clusters, where N_trucks = n_clusters
    grouped_coords is a list of lists containing coordinates, ex: grouped_coords[0] = list of all coordinates in cluster 0
    """
    pass

def sTSP(cost_matrix):
    """
    inputs: nparray cost_matrix
    outputs: list[tuple]
    description: calculates optimal sequence for a delivery truck
    """
    
def plot_route(travel_seq):
    """
    inputs: list[tuple] travel_seq
    outputs: none
    description: visualizes travel route on folium
    """